### Automated anomaly detection using PyOD and Auto-Sklearn
This notebook incorporates anomaly detection algorithms from PyOD into Auto-Sklearn. The following steps are followed:
- Auto-Sklearn is extended to incorporate PyOD algorithms
- Anomaly detection dataset(s) are imported and pre-processed
- Dataset is passed to Auto-Sklearn automl object for automated modeling and prediction
- Results and stats are obtained and analyzed

In [ ]:
# Imports
import pandas as pd
import numpy as np
import sklearn.metrics
import autosklearn.classification
from sklearn.model_selection import train_test_split
from autosklearn.pipeline.components.classification import add_classifier
from autosklearn.metrics import roc_auc, average_precision
from scipy.io import arff
from matplotlib import pyplot as plt

In [ ]:
# Shuttle dataset
from scipy.io import loadmat  # this is the SciPy module that loads mat-files
mat = loadmat('../../data/shuttle.mat')  # load mat-file
X = mat['X']
y = mat['y']
X = pd.DataFrame(X)
y = pd.DataFrame(y)
y = y[0]
N = 5000
if(len(X) > N):
    X = X.sample(n=N, ignore_index=True, random_state=1)
    y = y.sample(n=N, ignore_index=True, random_state=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1, stratify=y) # stratified split
# Reset indexes
X_train.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)
X_test.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)
print("Ratio of outliers in training set:", len(y_train[y_train==1])/len(y_train))
print("Ratio of outliers in test set:", len(y_test[y_test==1])/len(y_test))
print("Training size:", len(X_train))
print("Test size:", len(X_test))

In [ ]:
# Import Auto-Sklearn implemented PyOD classifiers
import os, sys
p = os.path.abspath('..')
sys.path.insert(1, p)
from pyod_models.abod import ABODClassifier # probabilistic
from pyod_models.cblof import CBLOFClassifier # proximity-based
from pyod_models.cof import COFClassifier # proximity-based
from pyod_models.copod import COPODClassifier # probabilistic
from pyod_models.ecod import ECODClassifier # probabilistic
from pyod_models.hbos import HBOSClassifier # proximity-based
from pyod_models.iforest import IForestClassifier # outlier ensembles
from pyod_models.knn import KNNClassifier # proximity-based
from pyod_models.lmdd import LMDDClassifier # linear model
from pyod_models.loci import LOCIClassifier # proximity-based
from pyod_models.lof import LOFClassifier # proximity-based
from pyod_models.mad import MADClassifier # probabilistic
from pyod_models.mcd import MCDClassifier # linear model
from pyod_models.ocsvm import OCSVMClassifier # linear model
from pyod_models.pca import PCAClassifier # linear model
from pyod_models.rod import RODClassifier # proximity-based
from pyod_models.sod import SODClassifier # proximity-based
from pyod_models.sos import SOSClassifier # probabilistic
# Add algorithms to the pipeline components of Auto-Sklearn
add_classifier(ABODClassifier)
add_classifier(CBLOFClassifier)
add_classifier(COFClassifier)
add_classifier(COPODClassifier)
add_classifier(ECODClassifier)
add_classifier(HBOSClassifier)
add_classifier(IForestClassifier)
add_classifier(KNNClassifier)
add_classifier(LMDDClassifier)
add_classifier(LOCIClassifier)
add_classifier(LOFClassifier)
add_classifier(MADClassifier)
add_classifier(MCDClassifier)
add_classifier(OCSVMClassifier)
add_classifier(PCAClassifier)
add_classifier(RODClassifier)
add_classifier(SODClassifier)
add_classifier(SOSClassifier)

In [ ]:
# Pre-defined split indices for train and validation
selected_indices = []
sacrificed_outliers = 0 # count of outliers sampled for the training set
for v in y_train:
    if v==1: # outlier
        if np.random.rand()>0.05:
            selected_indices.append(1) # validation
        else:
            selected_indices.append(-1) # training
            sacrificed_outliers += 1 # will not be used for evaluation
    else:
        if np.random.rand()>0.9:
            selected_indices.append(1) # validation
        else:
            selected_indices.append(-1) # training
# prints
print('Length of selected indices:', len(selected_indices))
print('Number of total samples:', len(y_train))
print('Number of outliers:', sum(y_train))
print('Number of sacrificed outliers:', sacrificed_outliers)
print('Number of validation samples:', selected_indices.count(1))
print('Number of training samples:', selected_indices.count(-1))

In [ ]:
# Build and fit a classifier
resampling_strategy = sklearn.model_selection.PredefinedSplit(test_fold=selected_indices)
automl = autosklearn.classification.AutoSklearnClassifier(
    include = {
      'classifier': [
          'ABODClassifier',
          'CBLOFClassifier',
          #'COFClassifier',
          'COPODClassifier',
          'ECODClassifier',
          'HBOSClassifier',
          'IForestClassifier',
          'KNNClassifier',
          'LMDDClassifier',
          'LOCIClassifier',
          'LOFClassifier',
          #'MADClassifier', # only for univariate data
          'MCDClassifier',
          'OCSVMClassifier',
          'PCAClassifier',
          'RODClassifier',
          'SODClassifier',
          'SOSClassifier'
      ],
      'feature_preprocessor': ["no_preprocessing"],
    },
    exclude=None,
    metric=roc_auc,
    scoring_functions = [ roc_auc, average_precision],
    time_left_for_this_task=60,
    per_run_time_limit=15,
    ensemble_size=1,
    initial_configurations_via_metalearning=0,
    delete_tmp_folder_after_terminate=False,
    resampling_strategy=resampling_strategy,
    #resampling_strategy_arguments={'train_size': 0.67}
    #resampling_strategy='cv',
    #resampling_strategy_arguments={'folds': 10}
)
# Fit
print('Running fit...')
automl.fit(X_train, y_train, X_test, y_test, dataset_name='Shuttle')

In [ ]:
# auto-sklearn execution details
print(automl.sprint_statistics())
# Top ranked model
#print(automl.leaderboard(top_k=10))
# Top ranked model configuration
#print()
print(automl.show_models())

In [ ]:
print('Running refit...')
automl.refit(X_train, y_train)
# Evaluate on test set
y_pred=automl.predict_proba(X_test)
# score
score=sklearn.metrics.roc_auc_score(y_test,y_pred[:,1])
print("ROC AUC score:", score)

In [ ]:
# Get metric results function definition
def get_metric_result(cv_results):
    results = pd.DataFrame.from_dict(cv_results)
    #results = results[results['status'] == "Success"] # if we're interested only in successful runs
    cols = ['rank_test_scores', 'status', 'param_classifier:__choice__', 'mean_test_score', 'mean_fit_time']
    cols.extend([key for key in cv_results.keys() if key.startswith('metric_')]) # if there are additional metrics
    return results[cols].sort_values(['rank_test_scores'])
# Call the function
print(get_metric_result(automl.cv_results_).to_string(index=False))

In [ ]:
# Plot training performance over time
automl.performance_over_time_.plot(
   x='Timestamp',
   kind='line',
   legend=True,
   title='Auto-sklearn ROC AUC score over time',
   grid=True,
)
plt.show()